# Variants discovery

## Introduction

The work was performed on Linux Ubuntu 22.04.3 LTS. List of software used:

|Title|Version|
|--|--|
|Jupyter Notebook|6.5.4|
|GNU bash|5.1.16(1)-release-(x86_64-pc-linux-gnu)|
|Python|3.11.6|
|pandas|2.1.4|
|BWA|0.7.17-r1188|
|GATK|4.4.0.0|
|Samtools|1.6|
|Picard|2.27.5|
|SnpEff|5.2|

FASTQ files with paired-end reads are given.

<ins>Purpose of the work</ins>: to perform an analysis of gene variants in the provided FASTQ files.

1. Install GATK version 4.4.0.0.
2. Download paired-end read data.
3. Install BWA.
4. Download and index the reference genome.
5. Perform paired-end reads alignment using BWA.
6. Label PCR duplicates.
7. Implement BQSR recalibration.
8. Perform variant calling.
9. Annotate the VCF file using SnpEff.
10. Filter results by quality parameters.
11. Describe the results obtained by answering the following questions:
* What genes have mutations?
* How many mutations are there on average in each gene?

## 1. Installing `GATK` using `conda`

In [ ]:
!conda install -c bioconda gatk4 -y

Solving environment: done

## Package Plan ##

  environment location: /home/snitkin/anaconda3/envs/variant

  added / updated specs:
    - gatk4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    gatk4-4.4.0.0              |   py36hdfd78af_0       346.8 MB  bioconda
    ------------------------------------------------------------
                                           Total:       346.8 MB

The following NEW packages will be INSTALLED:

  gatk4              bioconda/noarch::gatk4-4.4.0.0-py36hdfd78af_0 None



gatk4-4.4.0.0        | 346.8 MB  | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done


## 2. Downloading data

Downloaded FASTQ files and explored.

In [ ]:
!ls data -sh

итого 120M
58M TK44R8S18.R1.fq.gz	63M TK44R8S18.R2.fq.gz


In [ ]:
!gunzip data/TK44R8S18.R1.fq.gz -c | head

@M03474:50:000000000-B6V27:1:1109:7091:21898
TTGATAAACTCACCAAAGGCTTCCTTGGTATTTCAGACAGATTATGAGGGACAAGCCAAGAAACTCCTGGAACTGATGGAAAACACGGATGTGATCATTGTTGCAGGAGGAGATGGGACACTGCAGGAGGTATG
+
A@BBBCEEDEEEFCFDDFEFEDECEDFECDEDDEFFEEFFCEDDEFEFEDCEEEFEBEEFECEEEECEFEEEEEFEEFEEEECEFE<DCDFCFEEEFEDFCDFEFFECEDCEEEFEEEEEEEFEFFECEDBDEF
@M03474:50:000000000-B6V27:1:2102:17603:27125
AGGAGCTGCTGAAGATGTGGGGGGAGGAGCTGACCAGTGAAGCAAGTGTCTTTGAAGTCTTTGTTCTTTACCTCTCGGGAGAACCAAACCGGAATGGTCACAAAGTGAGTGATGCTGGAGTGGGGACCCTGGTTCATCCCCTGCCCCTGGCCTGACCCCAG
+
AA@B@?EFDEADDFDEGBGEEEDDADDCBCEFEECFFCFEEFEEEFCFCEECDFCCFCEEDDFBDEEDDDECEEEE<DDCECEECFEEEB<DCCDFDCEFEFEEFCFEFCFEEFEDFEEFCFBEDCEB:DEBBCEEDEBBBDFEBBBDFDEBDECCBBBDB
@M03474:50:000000000-B6V27:1:1106:18978:2648
GCATTGAATAATGACAGACTGAATAGATATGATCTGATTCATGTGAAATGTCTCTCTTCTCAGTCCTCAGGTGGAAGATAGTGAGGGGACTATTCAATTTGATGGAGAAGGTTATGCATTGGTCAGCCGTCCCATTCGCTGGTACCCCAACATCTCCACTG

gzip: stdout: Broken pipe


## 3. Installing `bwa` using `conda`

In [ ]:
!conda install -c bioconda bwa -y

Channels:
 - bioconda
 - defaults
 - anaconda
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/snitkin/anaconda3/envs/variant

  added / updated specs:
    - bwa


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bwa-0.7.17                 |       h5bf99c6_8         608 KB  bioconda
    ------------------------------------------------------------
                                           Total:         608 KB

The following NEW packages will be INSTALLED:

  bwa                bioconda/linux-64::bwa-0.7.17-h5bf99c6_8 
  gdbm               pkgs/main/linux-64::gdbm-1.18-hd4cb3f1_4 
  perl               pkgs/main/linux-64::perl-5.34.0-h5eee18b_2 



                                                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


## 4. Downloading and indexing the reference genome

In [ ]:
%%bash
mkdir genome
genome_file_name="Homo_sapiens_assembly38.fasta"
wget "https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/$genome_file_name" \
    -O "genome/$genome_file_name" --no-verbose

2023-12-14 23:52:10 URL:https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.fasta [3249912778/3249912778] -> "genome/Homo_sapiens_assembly38.fasta" [1]


In [ ]:
%%bash
genome_file_name="Homo_sapiens_assembly38.fasta"
bwa index "genome/$genome_file_name"

[bwa_index] Pack FASTA... 21.50 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=6434693834, availableWord=464768632
[BWTIncConstructFromPacked] 10 iterations done. 99999994 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 199999994 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 299999994 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 399999994 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 499999994 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 599999994 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 699999994 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 799999994 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 899999994 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 999999994 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 

## 5. Aligning reads to the genome

In [ ]:
%%bash
mkdir bwa
forward_reads="data/TK44R8S18.R1.fq.gz"
reverse_reads="data/TK44R8S18.R2.fq.gz"
group_header="@RG\tID:M03474\tSM:TK44R8S18\tPL:Illumina"
bwa mem -R $group_header -t 3 \
    "genome/Homo_sapiens_assembly38.fasta" $forward_reads $reverse_reads \
    | samtools sort -m 3G -> "bwa/TK44R8S18.sorted.bam"

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 202906 sequences (30000185 bp)...
[M::process] read 202936 sequences (30000087 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (8, 91674, 0, 10)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (163, 225, 288)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 538)
[M::mem_pestat] mean and std.dev: (229.30, 90.68)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 663)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation RR...
[M::mem_pestat] (25, 50, 75) percentile: (154, 188, 269)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 499)
[M::mem_pestat] mean and std.dev: (204.11, 75.98)
[M::mem_pestat] low and high boundaries for pro

## 6. Marking duplicates with `gatk MarkDuplicates`

In [ ]:
%%bash
bam_base="bwa/TK44R8S18.sorted"

gatk MarkDuplicates \
     --INPUT "$bam_base.bam" \
     --METRICS_FILE "$bam_base.marked.log" \
     --ASSUME_SORT_ORDER coordinate \
     --CREATE_INDEX true \
     --OUTPUT "$bam_base.marked.bam"

Using GATK jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar MarkDuplicates --INPUT bwa/TK44R8S18.sorted.bam --METRICS_FILE bwa/TK44R8S18.sorted.marked.log --ASSUME_SORT_ORDER coordinate --CREATE_INDEX true --OUTPUT bwa/TK44R8S18.sorted.marked.bam
15:59:47.610 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
[Fri Dec 15 15:59:47 MSK 2023] MarkDuplicates --INPUT bwa/TK44R8S18.sorted.bam --OUTPUT bwa/TK44R8S18.sorted.marked.bam --METRICS_FILE bwa/TK44R8S18.sorted.marked.log --ASSUME_SORT_ORDER coordinate --CREATE_INDEX true --M

Tool returned:
0


## 7. BQSR recalibration

Downloaded the gold standard in variant calling [HapMap](http://hapmap.ncbi.nlm.nih.gov/).

In [ ]:
%%bash
mkdir hapmap
cd hapmap

wget https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/hapmap_3.3.hg38.vcf.gz
wget https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/hapmap_3.3.hg38.vcf.gz.tbi

--2023-12-15 13:06:07--  https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/hapmap_3.3.hg38.vcf.gz
Распознаётся storage.googleapis.com (storage.googleapis.com)… 2a00:1450:4026:803::201b, 2a00:1450:4026:808::201b, 2a00:1450:4026:805::201b, ...
Подключение к storage.googleapis.com (storage.googleapis.com)|2a00:1450:4026:803::201b|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 62043448 (59M) [text/x-vcard]
Сохранение в: ‘hapmap_3.3.hg38.vcf.gz’

     0K .......... .......... .......... .......... ..........  0%  269K 3m45s
    50K .......... .......... .......... .......... ..........  0%  326K 3m25s
   100K .......... .......... .......... .......... ..........  0% 1,06M 2m35s
   150K .......... .......... .......... .......... ..........  0%  764K 2m16s
   200K .......... .......... .......... .......... ..........  0% 2,53M 1m53s
   250K .......... .......... .......... .......... ..........  0% 1,40M 1m41s
   300K .......

  5000K .......... .......... .......... .......... ..........  8% 1,60M 41s
  5050K .......... .......... .......... .......... ..........  8% 2,02M 41s
  5100K .......... .......... .......... .......... ..........  8% 1,16M 41s
  5150K .......... .......... .......... .......... ..........  8% 1,68M 41s
  5200K .......... .......... .......... .......... ..........  8% 1,08M 41s
  5250K .......... .......... .......... .......... ..........  8% 1,21M 41s
  5300K .......... .......... .......... .......... ..........  8% 1,61M 41s
  5350K .......... .......... .......... .......... ..........  8% 1,60M 41s
  5400K .......... .......... .......... .......... ..........  8% 2,00M 41s
  5450K .......... .......... .......... .......... ..........  9% 1,71M 41s
  5500K .......... .......... .......... .......... ..........  9% 1,30M 41s
  5550K .......... .......... .......... .......... ..........  9% 1,40M 41s
  5600K .......... .......... .......... .......... ..........  9%  834K 41s

 10350K .......... .......... .......... .......... .......... 17% 1,97M 35s
 10400K .......... .......... .......... .......... .......... 17% 1,66M 35s
 10450K .......... .......... .......... .......... .......... 17% 5,39M 35s
 10500K .......... .......... .......... .......... .......... 17% 3,24M 34s
 10550K .......... .......... .......... .......... .......... 17% 2,02M 34s
 10600K .......... .......... .......... .......... .......... 17% 1,71M 34s
 10650K .......... .......... .......... .......... .......... 17% 3,65M 34s
 10700K .......... .......... .......... .......... .......... 17% 4,89M 34s
 10750K .......... .......... .......... .......... .......... 17% 2,02M 34s
 10800K .......... .......... .......... .......... .......... 17% 2,18M 34s
 10850K .......... .......... .......... .......... .......... 17% 2,67M 34s
 10900K .......... .......... .......... .......... .......... 18% 3,50M 34s
 10950K .......... .......... .......... .......... .......... 18% 2,75M 33s

 15700K .......... .......... .......... .......... .......... 25% 1,63M 26s
 15750K .......... .......... .......... .......... .......... 26% 2,49M 26s
 15800K .......... .......... .......... .......... .......... 26% 2,62M 26s
 15850K .......... .......... .......... .......... .......... 26% 3,31M 26s
 15900K .......... .......... .......... .......... .......... 26% 3,65M 26s
 15950K .......... .......... .......... .......... .......... 26% 1,86M 26s
 16000K .......... .......... .......... .......... .......... 26% 1,86M 26s
 16050K .......... .......... .......... .......... .......... 26% 3,49M 26s
 16100K .......... .......... .......... .......... .......... 26% 1,21M 26s
 16150K .......... .......... .......... .......... .......... 26% 4,76M 26s
 16200K .......... .......... .......... .......... .......... 26% 1,99M 26s
 16250K .......... .......... .......... .......... .......... 26% 1,92M 26s
 16300K .......... .......... .......... .......... .......... 26% 2,35M 26s

 21050K .......... .......... .......... .......... .......... 34% 6,16M 21s
 21100K .......... .......... .......... .......... .......... 34% 4,67M 21s
 21150K .......... .......... .......... .......... .......... 34% 2,73M 21s
 21200K .......... .......... .......... .......... .......... 35% 2,57M 21s
 21250K .......... .......... .......... .......... .......... 35% 3,83M 21s
 21300K .......... .......... .......... .......... .......... 35% 2,24M 21s
 21350K .......... .......... .......... .......... .......... 35% 2,65M 21s
 21400K .......... .......... .......... .......... .......... 35% 2,50M 21s
 21450K .......... .......... .......... .......... .......... 35% 12,0M 21s
 21500K .......... .......... .......... .......... .......... 35% 3,10M 21s
 21550K .......... .......... .......... .......... .......... 35% 3,05M 21s
 21600K .......... .......... .......... .......... .......... 35% 2,42M 21s
 21650K .......... .......... .......... .......... .......... 35% 4,12M 21s

...

 37100K .......... .......... .......... .......... .......... 61% 5,47M 10s
 37150K .......... .......... .......... .......... .......... 61% 2,78M 10s
 37200K .......... .......... .......... .......... .......... 61% 2,64M 10s
 37250K .......... .......... .......... .......... .......... 61% 3,74M 10s
 37300K .......... .......... .......... .......... .......... 61% 4,51M 10s
 37350K .......... .......... .......... .......... .......... 61% 2,60M 10s
 37400K .......... .......... .......... .......... .......... 61% 2,54M 10s
 37450K .......... .......... .......... .......... .......... 61% 1,82M 10s
 37500K .......... .......... .......... .......... .......... 61% 3,44M 10s
 37550K .......... .......... .......... .......... .......... 62% 2,71M 10s
 37600K .......... .......... .......... .......... .......... 62% 1,64M 10s
 37650K .......... .......... .......... .......... .......... 62% 4,34M 10s
 37700K .......... .......... .......... .......... .......... 62% 2,13M 10s

 42500K .......... .......... .......... .......... .......... 70% 4,50M 7s
 42550K .......... .......... .......... .......... .......... 70% 2,27M 7s
 42600K .......... .......... .......... .......... .......... 70% 5,28M 7s
 42650K .......... .......... .......... .......... .......... 70% 2,61M 7s
 42700K .......... .......... .......... .......... .......... 70% 4,86M 7s
 42750K .......... .......... .......... .......... .......... 70% 4,93M 7s
 42800K .......... .......... .......... .......... .......... 70% 2,30M 7s
 42850K .......... .......... .......... .......... .......... 70% 2,38M 7s
 42900K .......... .......... .......... .......... .......... 70% 2,19M 7s
 42950K .......... .......... .......... .......... .......... 70% 3,49M 7s
 43000K .......... .......... .......... .......... .......... 71% 6,43M 7s
 43050K .......... .......... .......... .......... .......... 71% 3,71M 7s
 43100K .......... .......... .......... .......... .......... 71% 2,69M 7s
 43150K ....

 47900K .......... .......... .......... .......... .......... 79% 2,20M 5s
 47950K .......... .......... .......... .......... .......... 79%  120K 5s
 48000K .......... .......... .......... .......... .......... 79% 60,7M 5s
 48050K .......... .......... .......... .......... .......... 79% 68,9M 5s
 48100K .......... .......... .......... .......... .......... 79% 67,8M 5s
 48150K .......... .......... .......... .......... .......... 79% 69,6M 5s
 48200K .......... .......... .......... .......... .......... 79% 69,3M 5s
 48250K .......... .......... .......... .......... .......... 79%  112M 5s
 48300K .......... .......... .......... .......... .......... 79% 73,3M 5s
 48350K .......... .......... .......... .......... .......... 79% 96,7M 5s
 48400K .......... .......... .......... .......... .......... 79% 58,6M 5s
 48450K .......... .......... .......... .......... .......... 80% 76,2M 5s
 48500K .......... .......... .......... .......... .......... 80% 89,6M 5s
 48550K ....

 53300K .......... .......... .......... .......... .......... 88% 4,85M 3s
 53350K .......... .......... .......... .......... .......... 88% 3,82M 3s
 53400K .......... .......... .......... .......... .......... 88% 12,3M 3s
 53450K .......... .......... .......... .......... .......... 88% 3,47M 3s
 53500K .......... .......... .......... .......... .......... 88% 4,45M 3s
 53550K .......... .......... .......... .......... .......... 88% 5,88M 3s
 53600K .......... .......... .......... .......... .......... 88% 3,46M 3s
 53650K .......... .......... .......... .......... .......... 88% 3,16M 3s
 53700K .......... .......... .......... .......... .......... 88% 10,3M 3s
 53750K .......... .......... .......... .......... .......... 88% 4,50M 3s
 53800K .......... .......... .......... .......... .......... 88% 7,29M 3s
 53850K .......... .......... .......... .......... .......... 88% 4,40M 3s
 53900K .......... .......... .......... .......... .......... 89% 3,21M 3s
 53950K ....

 58700K .......... .......... .......... .......... .......... 96% 2,08M 1s
 58750K .......... .......... .......... .......... .......... 97% 3,73M 1s
 58800K .......... .......... .......... .......... .......... 97% 5,04M 1s
 58850K .......... .......... .......... .......... .......... 97% 1,85M 1s
 58900K .......... .......... .......... .......... .......... 97% 1,87M 1s
 58950K .......... .......... .......... .......... .......... 97% 1,73M 1s
 59000K .......... .......... .......... .......... .......... 97% 2,61M 1s
 59050K .......... .......... .......... .......... .......... 97% 3,90M 1s
 59100K .......... .......... .......... .......... .......... 97% 2,58M 1s
 59150K .......... .......... .......... .......... .......... 97% 3,51M 1s
 59200K .......... .......... .......... .......... .......... 97% 3,10M 1s
 59250K .......... .......... .......... .......... .......... 97% 3,47M 1s
 59300K .......... .......... .......... .......... .......... 97% 4,77M 0s
 59350K ....

In [ ]:
%%bash
fasta_base="genome/Homo_sapiens_assembly38"
samtools faidx $fasta_base.fasta
picard CreateSequenceDictionary -R $fasta_base.fasta -O $fasta_base.dict

13:49:31.781 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/dimasnitkin/miniconda3/share/picard-2.27.5-0/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Fri Dec 15 13:49:31 MSK 2023] CreateSequenceDictionary --OUTPUT Homo_sapiens_assembly38.dict --REFERENCE Homo_sapiens_assembly38.fasta --TRUNCATE_NAMES_AT_WHITESPACE true --NUM_SEQUENCES 2147483647 --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 5 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --GA4GH_CLIENT_SECRETS client_secrets.json --help false --version false --showHidden false --USE_JDK_DEFLATER false --USE_JDK_INFLATER false
[Fri Dec 15 13:49:31 MSK 2023] Executing as dimasnitkin@cluster.img.ras.ru on Linux 3.10.0-957.el7.x86_64 amd64; OpenJDK 64-Bit Server VM 1.8.0_152-release-1056-b12; Deflater: Intel; Inflater: Intel; Provider GCS is not available; Picard version: Version:2.27.5
[Fri Dec 15 13:50:00 MSK 2023] picard.sam.CreateSe

In [ ]:
%%bash
mkdir gatk
reference="genome/Homo_sapiens_assembly38.fasta"
known_sites="hapmap/hapmap_3.3.hg38.vcf.gz"

gatk BaseRecalibrator \
    --input "bwa/TK44R8S18.sorted.marked.bam" \
    --reference $reference \
    --known-sites $known_sites \
    --output "gatk/TK44R8S18.recalib.table"

Using GATK jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar BaseRecalibrator --input bwa/TK44R8S18.sorted.marked.bam --reference genome/Homo_sapiens_assembly38.fasta --known-sites hapmap/hapmap_3.3.hg38.vcf.gz --output gatk/TK44R8S18.recalib.table
16:03:01.748 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
16:03:01.782 INFO  BaseRecalibrator - ------------------------------------------------------------
16:03:01.786 INFO  BaseRecalibrator - The Genome Analysis Toolkit (GATK) v4.4.0.0
16:03:01.786 INFO  BaseRecalibrator - F

Tool returned:
SUCCESS


In [ ]:
%%bash
bam_base="bwa/TK44R8S18.sorted.marked"

gatk ApplyBQSR \
    --reference "genome/Homo_sapiens_assembly38.fasta" \
    --input "$bam_base.bam" \
    --bqsr-recal-file "gatk/TK44R8S18.recalib.table" \
    --output "$bam_base.bqsr.bam"

Using GATK jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar ApplyBQSR --reference genome/Homo_sapiens_assembly38.fasta --input bwa/TK44R8S18.sorted.marked.bam --bqsr-recal-file gatk/TK44R8S18.recalib.table --output bwa/TK44R8S18.sorted.marked.bqsr.bam
16:05:17.551 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
16:05:17.593 INFO  ApplyBQSR - ------------------------------------------------------------
16:05:17.597 INFO  ApplyBQSR - The Genome Analysis Toolkit (GATK) v4.4.0.0
16:05:17.597 INFO  ApplyBQSR - For support and do

## 8. Variant calling

In [ ]:
%%bash
bam_base="bwa/TK44R8S18.sorted.marked.bqsr"

gatk --java-options "-Xmx4g" HaplotypeCaller  \
    -R "genome/Homo_sapiens_assembly38.fasta" \
    -I "$bam_base.bam" \
    -O "gatk/TK44R8S18.vcf.gz" \
    -bamout "$bam_base.variants.bam"

Using GATK jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -Xmx4g -jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar HaplotypeCaller -R genome/Homo_sapiens_assembly38.fasta -I bwa/TK44R8S18.sorted.marked.bqsr.bam -O gatk/TK44R8S18.vcf.gz -bamout bwa/TK44R8S18.sorted.marked.bqsr.variants.bam
16:09:05.105 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
16:09:05.147 INFO  HaplotypeCaller - ------------------------------------------------------------
16:09:05.150 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.4.0.0
16:09:05.151 INFO  HaplotypeCaller - For suppor

16:16:05.878 INFO  ProgressMeter -       chr13:90753077              7.0               7251490        1035678.1
16:16:15.878 INFO  ProgressMeter -       chr14:35668488              7.2               7449560        1039229.4
16:16:25.878 INFO  ProgressMeter -       chr14:85924494              7.3               7617720        1038541.6
16:16:35.885 INFO  ProgressMeter -       chr15:39961485              7.5               7821730        1042647.1
16:16:45.888 INFO  ProgressMeter -       chr15:89418746              7.7               7987280        1041567.8
16:16:55.888 INFO  ProgressMeter -       chr16:37365765              7.8               8154430        1040745.3
16:17:05.907 INFO  ProgressMeter -       chr16:85134376              8.0               8314190        1038992.4
16:17:15.907 INFO  ProgressMeter -       chr17:42801550              8.2               8474890        1037466.4
16:17:25.907 INFO  ProgressMeter -        chr18:6529906              8.3               8632210        10

## 9. Annotation of VCF file with `SnpEff`

In [ ]:
%%bash
java -Xmx8g -jar snpEff/snpEff.jar -verbose GRCh38.86 "gatk/TK44R8S18.vcf.gz" > "gatk/TK44R8S18.ann.vcf"

00:00:00 SnpEff version SnpEff 5.2a (build 2023-10-24 14:24), by Pablo Cingolani
00:00:00 Command: 'ann'
00:00:00 Reading configuration file 'snpEff.config'. Genome: 'GRCh38.86'
00:00:00 Reading config file: /state/partition1/raid10/workspaces/dimasnitkin/human/variant/snpEff.config
00:00:00 Reading config file: /state/partition1/raid10/workspaces/dimasnitkin/human/variant/snpEff/snpEff.config
00:00:01 done
00:00:01 Reading database for genome version 'GRCh38.86' from file '/state/partition1/raid10/workspaces/dimasnitkin/human/variant/snpEff/./data/GRCh38.86/snpEffectPredictor.bin' (this might take a while)
00:00:01 Database not installed
	Attempting to download and install database 'GRCh38.86'
00:00:01 Reading configuration file 'snpEff.config'. Genome: 'GRCh38.86'
00:00:01 Reading config file: /state/partition1/raid10/workspaces/dimasnitkin/human/variant/snpEff.config
00:00:01 Reading config file: /state/partition1/raid10/workspaces/dimasnitkin/human/variant/snpEff/snpEff.config
00:0

#		'3'	198295559	Standard
#		'HG2066_PATCH'	198295555	Standard
#		'HSCHR3_6_CTG3'	198294921	Standard
#		'HSCHR3_7_CTG3'	198294497	Standard
#		'HSCHR3_5_CTG3'	198293785	Standard
#		'HSCHR3_3_CTG3'	198293223	Standard
#		'HSCHR3_9_CTG3'	198291952	Standard
#		'HG2237_PATCH'	198290329	Standard
#		'HSCHR4_11_CTG12'	190233769	Standard
#		'HSCHR4_1_CTG12'	190227204	Standard
#		'HSCHR4_7_CTG12'	190225141	Standard
#		'HSCHR4_1_CTG6'	190224838	Standard
#		'HSCHR4_1_CTG4'	190221757	Standard
#		'HSCHR4_4_CTG12'	190220704	Standard
#		'HSCHR4_5_CTG12'	190219562	Standard
#		'HSCHR4_9_CTG12'	190219005	Standard
#		'HSCHR4_1_CTG8_1'	190218838	Standard
#		'HSCHR4_6_CTG12'	190218533	Standard
#		'HSCHR4_8_CTG12'	190218480	Standard
#		'HSCHR4_3_CTG12'	190218028	Standard
#		'HSCHR4_2_CTG4'	190217898	Standard
#		'4'	190214555	Standard
#		'HSCHR4_2_CTG12'	190206797	Standard
#		'HG2023_PATCH'	190147504	Standard
#		'HSCHR4_1_CTG9'	190096932	Standard
#		'HSCHR5_1_CTG1'	181558402	Standard
#		'HSCHR5_3_CTG1_1'	18155

#		'HSCHR18_ALT2_CTG2_1'	80376883	Standard
#		'HSCHR18_ALT21_CTG2_1'	80376343	Standard
#		'18'	80373285	Standard
#		'HSCHR20_1_CTG1'	64450183	Standard
#		'HSCHR20_1_CTG3'	64447812	Standard
#		'HSCHR20_1_CTG4'	64446453	Standard
#		'20'	64444167	Standard
#		'HSCHR20_1_CTG2'	64441059	Standard
#		'HSCHR19KIR_FH15_B_HAP_CTG3_1'	58689284	Standard
#		'HSCHR19KIR_FH05_B_HAP_CTG3_1'	58689016	Standard
#		'HSCHR19KIR_FH13_BA2_HAP_CTG3_1'	58662330	Standard
#		'HSCHR19KIR_RSH_BA2_HAP_CTG3_1'	58662237	Standard
#		'HSCHR19KIR_G248_BA2_HAP_CTG3_1'	58661364	Standard
#		'HSCHR19LRC_LRC_J_CTG3_1'	58650772	Standard
#		'HSCHR19KIR_FH08_BAX_HAP_CTG3_1'	58647709	Standard
#		'HSCHR19KIR_GRC212_AB_HAP_CTG3_1'	58644315	Standard
#		'HG2021_PATCH'	58632883	Standard
#		'HSCHR19KIR_LUCE_BDEL_HAP_CTG3_1'	58631446	Standard
#		'HSCHR19KIR_RP5_B_HAP_CTG3_1'	58631402	Standard
#		'HSCHR19KIR_T7526_BDEL_HAP_CTG3_1'	58631206	Standard
#		'HSCHR19_2_CTG2'	58627095	Standard
#		'HSCHR19_3_CTG3_1'	58625958	Standard
#		'HSCHR19_

#		'KI270412.1'	1179	Standard
#		'KI270395.1'	1143	Standard
#		'KI270376.1'	1136	Standard
#		'KI270337.1'	1121	Standard
#		'KI270335.1'	1048	Standard
#		'KI270378.1'	1048	Standard
#		'KI270379.1'	1045	Standard
#		'KI270329.1'	1040	Standard
#		'KI270419.1'	1029	Standard
#		'KI270336.1'	1026	Standard
#		'KI270312.1'	998	Standard
#		'KI270539.1'	993	Standard
#		'KI270385.1'	990	Standard
#		'KI270423.1'	981	Standard
#		'KI270392.1'	971	Standard
#		'KI270394.1'	970	Standard
#-----------------------------------------------

00:13:28 Predicting variants
00:13:28 Loading sequences for chromosome '1' from file '/state/partition1/raid10/workspaces/dimasnitkin/human/variant/snpEff/./data/GRCh38.86/sequence.1.bin'
00:13:32 Building sequence tree for chromosome '1'
00:13:32 Done. Loaded 3134 sequences.
00:13:33 Loading sequences for chromosome '2' from file '/state/partition1/raid10/workspaces/dimasnitkin/human/variant/snpEff/./data/GRCh38.86/sequence.2.bin'
00:13:37 Building sequence tree for chro

SnpEff produces an HTML file with a fairly detailed summary of the annotation. Obviously, most mutations occur in introns, but I would not have thought that significantly more than in intergenic regions!

In [ ]:
from IPython.display import HTML

HTML(filename='snpEff/snpEff_summary.html')

## 10. Filtration of the annotated VCF file by quality

I used filters from [the GATK webpage](https://gatk.broadinstitute.org/hc/en-us/articles/360035890471-Hard-filtering-germline-short-variants).

In [ ]:
%%bash

vcf_base="gatk/TK44R8S18.ann"
gatk VariantFiltration \
    -V "$vcf_base.vcf" \
    -filter "QD < 2.0" --filter-name "QD2" \
    -filter "QUAL < 30.0" --filter-name "QUAL30" \
    -filter "SOR > 3.0" --filter-name "SOR3" \
    -filter "FS > 60.0" --filter-name "FS60" \
    -filter "MQ < 40.0" --filter-name "MQ40" \
    -filter "MQRankSum < -12.5" --filter-name "MQRankSum-12.5" \
    -filter "ReadPosRankSum < -8.0" --filter-name "ReadPosRankSum-8" \
    -O "$vcf_base.filtered.vcf"

Using GATK jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar VariantFiltration -V gatk/TK44R8S18.ann.vcf -filter QD < 2.0 --filter-name QD2 -filter QUAL < 30.0 --filter-name QUAL30 -filter SOR > 3.0 --filter-name SOR3 -filter FS > 60.0 --filter-name FS60 -filter MQ < 40.0 --filter-name MQ40 -filter MQRankSum < -12.5 --filter-name MQRankSum-12.5 -filter ReadPosRankSum < -8.0 --filter-name ReadPosRankSum-8 -O gatk/TK44R8S18.ann.filtered.vcf
17:35:45.686 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
17:35:45.723 INFO  Variant

17:35:46.206 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.208 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.208 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.209 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.210 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.212 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.212 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.213 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.214 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.216 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.216 WARN  JexlEngine - ![0,14]: 'ReadPosR

17:35:46.274 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.275 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.276 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.276 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.277 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.277 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.278 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.278 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.279 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.280 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.281 WARN  JexlEngine - ![0,9]: 'MQRankSum

17:35:46.323 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.323 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.324 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.325 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.325 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.326 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.326 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.327 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.327 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.328 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.328 WARN  JexlEngine - ![0,14]: 'ReadPosR

17:35:46.371 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.371 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.372 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.372 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.373 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.373 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.373 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.374 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.374 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:46.374 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:46.375 WARN  JexlEngine - ![0,9]: 'MQRankSum

...

17:35:51.154 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.154 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.154 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.154 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.154 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.155 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.155 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.155 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.155 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.155 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.155 WARN  JexlEngine - ![0,9]: 'MQRankSum

17:35:51.156 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.156 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.156 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.156 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.156 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.156 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.156 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.156 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.156 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.157 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.157 WARN  JexlEngine - ![0,14]: 'ReadPosR

17:35:51.325 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.325 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.325 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.325 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.325 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.325 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.325 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.325 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.325 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.325 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.325 WARN  JexlEngine - ![0,14]: 'ReadPosR

17:35:51.327 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.328 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.328 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.328 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.328 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.328 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.328 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.328 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.328 WARN  JexlEngine - ![0,9]: 'MQRankSum < -12.5;' undefined variable MQRankSum
17:35:51.328 WARN  JexlEngine - ![0,14]: 'ReadPosRankSum < -8.0;' undefined variable ReadPosRankSum
17:35:51.328 WARN  JexlEngine - ![0,9]: 'MQRankSum

17:35:51.502 INFO  VariantFiltration - Shutting down engine
[15 декабря 2023 г., 17:35:51 MSK] org.broadinstitute.hellbender.tools.walkers.filters.VariantFiltration done. Elapsed time: 0.10 minutes.
Runtime.totalMemory()=210763776


## 11. Results

Converted VCF file to TSV table.

In [ ]:
%%bash
vcf_base="gatk/TK44R8S18.ann.filtered"

gatk VariantsToTable --variant "$vcf_base.vcf" --output "$vcf_base.tsv"

Using GATK jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar VariantsToTable --variant gatk/TK44R8S18.ann.filtered.vcf --output gatk/TK44R8S18.ann.filtered.tsv
18:09:45.354 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/snitkin/anaconda3/envs/variant/share/gatk4-4.4.0.0-0/gatk-package-4.4.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
18:09:45.388 INFO  VariantsToTable - ------------------------------------------------------------
18:09:45.390 INFO  VariantsToTable - The Genome Analysis Toolkit (GATK) v4.4.0.0
18:09:45.391 INFO  VariantsToTable - For support and documentation go to https://software.broadinstitute.org/gatk/
18:09:45.391 IN

Loaded with `pandas`.

In [ ]:
import re
import pandas as pd

df = pd.read_table('gatk/TK44R8S18.ann.filtered.tsv')
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,AC,AF,AN,ANN,BaseQRankSum,DP,ExcessHet,FS,InbreedingCoeff,LOF,MLEAC,MLEAF,MQ,MQRankSum,NMD,QD,ReadPosRankSum,SOR,TK44R8S18.GT,TK44R8S18.AD,TK44R8S18.DP,TK44R8S18.GQ,TK44R8S18.PL
0,chr1,1359548,.,A,G,37.32,PASS,2,1.00,2,G|upstream_gene_variant|MODIFIER|MXRA8|ENSG000...,NaN,2,0.0,0.0,NaN,NaN,1,0.500,60.00,NaN,NaN,18.66,NaN,0.693,G/G,"0,2",2,6,"49,6,0"
1,chr1,2302812,.,A,G,73.84,PASS,2,1.00,2,G|upstream_gene_variant|MODIFIER|SKI|ENSG00000...,NaN,3,0.0,0.0,NaN,NaN,1,0.500,60.00,NaN,NaN,24.61,NaN,2.833,G/G,"0,3",3,9,"87,9,0"
2,chr1,2323471,.,T,C,37.32,PASS,2,1.00,2,C|downstream_gene_variant|MODIFIER|RP4-713A8.1...,NaN,2,0.0,0.0,NaN,NaN,1,0.500,60.00,NaN,NaN,18.66,NaN,0.693,C/C,"0,2",2,6,"49,6,0"
3,chr1,3068978,.,AG,A,45.28,PASS,2,1.00,2,A|upstream_gene_variant|MODIFIER|PRDM16|ENSG00...,NaN,2,0.0,0.0,NaN,NaN,1,0.500,60.00,NaN,NaN,22.64,NaN,2.303,A/A,"0,2",2,6,"57,6,0"
4,chr1,3319425,.,AG,A,52.28,PASS,2,1.00,2,A|intron_variant|MODIFIER|PRDM16|ENSG000001426...,NaN,2,0.0,0.0,NaN,NaN,1,0.500,60.00,NaN,NaN,26.14,NaN,0.693,A/A,"0,2",2,6,"64,6,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8524,chrUn_JTFH01001973v1_decoy,1198,.,C,G,64.64,PASS,1,0.500,2,G||MODIFIER|||||||||||||ERROR_CHROMOSOME_NOT_F...,0.0,6,0.0,0.0,NaN,NaN,1,0.500,49.73,0.0,NaN,10.77,1.834,0.693,C/G,"4,2",6,72,"72,0,162"
8525,chrUn_JTFH01001973v1_decoy,1200,.,A,C,64.64,PASS,1,0.500,2,C||MODIFIER|||||||||||||ERROR_CHROMOSOME_NOT_F...,0.0,6,0.0,0.0,NaN,NaN,1,0.500,49.73,0.0,NaN,10.77,1.834,0.693,A/C,"4,2",6,72,"72,0,162"
8526,chrUn_JTFH01001973v1_decoy,1216,.,T,C,174.97,PASS,2,1.00,2,C||MODIFIER|||||||||||||ERROR_CHROMOSOME_NOT_F...,NaN,6,0.0,0.0,NaN,NaN,2,1.00,49.73,NaN,NaN,29.16,NaN,0.693,C/C,"0,6",6,18,"189,18,0"
8527,chrUn_JTFH01001987v1_decoy,1307,.,C,T,35.48,PASS,2,1.00,2,T||MODIFIER|||||||||||||ERROR_CHROMOSOME_NOT_F...,NaN,1,0.0,0.0,NaN,NaN,1,0.500,57.00,NaN,NaN,33.25,NaN,1.609,T/T,"0,1",1,3,"45,3,0"


### What genes have mutations?

I got a list of gene identifiers for each variant in the dataframe and wrote it in the `ensembl_ids` column. If there are two intersecting genes for one variant, I write their identifiers separated by commas.

In [ ]:
genes_list = []

for index, row in df.iterrows():
    genes = re.findall('ENSG\d+', row['ANN'])
    genes = ','.join(set(genes))
    genes_list.append(genes)

df['ensembl_ids'] = genes_list

variant_counts = df['ensembl_ids'].value_counts()
variant_counts

ensembl_ids
                                   207
ENSG00000131018                    107
ENSG00000130635                     94
ENSG00000198626                     76
ENSG00000196218                     60
                                  ... 
ENSG00000164989                      1
ENSG00000237153,ENSG00000173068      1
ENSG00000178031                      1
ENSG00000137145                      1
ENSG00000162576                      1
Name: count, Length: 3901, dtype: int64

Top 4 genes with the most variants:

|Ensebml ID|Gene symbol|Variants count|Name|Associated diseases|
|--|--|--|--|--|
|ENSG00000131018|SYNE1|107|spectrin repeat containing nuclear envelope protein 1|autosomal recessive spinocerebellar ataxia 8, also referred to as autosomal recessive cerebellar ataxia type 1 or recessive ataxia of Beauce|
|ENSG00000130635|COL5A1|94|collagen type V alpha 1 chain|Ehlers-Danlos syndrome, types I and II|
|ENSG00000198626|RYR2|76|ryanodine receptor 2|stress-induced polymorphic ventricular tachycardia and arrhythmogenic right ventricular dysplasia|
|ENSG00000196218|RYR1|60|ryanodine receptor 1|malignant hyperthermia susceptibility, central core disease, and minicore myopathy with external ophthalmoplegia|

### How many mutations are there on average in each gene?

On average, there are slightly more than two mutations per gene.

In [ ]:
round(np.mean(df['ensembl_ids'].value_counts()), 2)

2.19